# 090 Face Orientation Estimation Assignment

> COM6018

*Copyright &copy; 2023, 2024 Jon Barker, University of Sheffield. All rights reserved*.


## 1. Introduction

This notebook have been written to accompany the Face Orientation Estimation assignment. The notebook is unlike the previous labs notebooks, in that it is not a step-by-step guide to a solution. Instead, it contains some notes and snippets of code that you may find useful.


## 2. Understanding the data provided

The assignment makes use of a number of data files that you will need to download from the following location:

https://drive.google.com/drive/folders/10y3e2zKkh0lVpRZ3WC21Uu-v-EcbBSYs?usp=sharing

As described in the assignment handout, you are provided with the following files:

- `train.full.joblib` - the full training dataset
- `train.small.joblib` - a smaller subset of the training dataset
- `eval1.joblib` - a dataset for evaluating your model
- `knn.30.joblib`, `knn.50.joblib`, `knn.90.joblib` - pre-trained kNN models, i.e., a baseline solution

If you have not downloaded these already, do so now and store them in the same directory as this notebook.

### 2.1 The training data

We will first look at the training data. The data can be loaded using the `joblib` library. The training data is stored as a tuple with two elements: a NumPy array of face images and a NumPy array of labels. The code below loads the data. It is also printing out the shape of the arrays to confirm that it has loaded correctly.


In [ ]:
import joblib
images, face_id = joblib.load('train.small.joblib')
print(images.shape, face_id.shape)

Note, the shape of the image arrays is (2500, 125, 94). This means it is a 3-D array storing 2500 images, each of which is 125 pixels high and 94 pixels wide. We can extract a single image using `one_image = images(index)`. The code below extracts the $n$th image and displays it along with the corresponding person ID.

In [ ]:
import matplotlib.pyplot as plt
n = 200
plt.matshow(images[n], cmap='gray')  # You can also use 'imshow'
print(f'This is an image of person {face_id[n]}')

## 2.2 The evaluation data

The evaluation data is stored in a nested dictionary structure. The top level dictionary has a key that is the sub-image size (i.e, 30, 50, 90). This can be used to retrieve a dictionary that contains the image and label data, which are called `x_test` and `y_test` to be consistent with the conventions that we've used earlier (i.e., `x` is the input data, and `y` is the output label). The image data (`x_test`) is stored as a 2D NumPy array where each row represents a complete reshaped image. i.e., the set of N-by-N pixel images have been reshaped into rows of N-squared pixels.


The code below loads the data and displays the shape of the arrays for the 50 pixel subimages.

In [ ]:
import joblib
eval_data = joblib.load('eval1.joblib')
eval_50 = eval_data[50]
x_test = eval_50['x_test']
y_test = eval_50['y_test']
print(x_test.shape, y_test.shape)


Note that there are 2000 examples in the evaluation set. This breaks down into 500 examples of each of the four orientations. This is a relatively large evaluation set for this task and will enable you to get a fairly accurate estimate of the performance of your model.

The code below selects the $n$th images and displays it by reshaping it back into a 2D array and then using the `imshow` function.

In [ ]:
n = 140
sub_image = x_test[n].reshape(50, 50)
plt.matshow(sub_image, cmap='gray')  # You can also use 'imshow'
print(f'This image is in orientation: {y_test[n]}')

## 2.3 The baseline models

A set of baseline KNN models have been trained for you and stored in the files `knn.30.joblib`, `knn.50.joblib`, `knn.90.joblib`. These can be loaded using the `joblib` library. The code below loads the 50 pixel model and displays the accuracy on the evaluation set.


In [ ]:
import joblib

model = joblib.load('knn.50.joblib')

print(model)


Note above that the print statement provides a description of the model. You can see that it is Pipeline starting with a PCA step that reduces the dimensionality of the data to 50 components. This is followed by a KNN classifier with $k=1$. The performance of this model is not great and it should be easy for you to improve on it.

We can check the performance by using the model's score method and passing the evaluation data as follows,

In [ ]:
percent_correct = model.score(x_test, y_test) * 100 
print(f'The classifier is {percent_correct:.2f}% correct')

Or is we want to look in more detail, we can compute and display a confusion matrix as follows,

In [ ]:
# dispaly the confusion matrix using sklearn's display function

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 90, 180, 270])
disp.plot()




Study the above confusion matrix carefully. What do you notice about the errors? Are they random or is there a pattern to them? If there is a pattern, can you explain it?

## Measuring Human Performance

For tasks like this, it is interesting to be able to compare the model performance against Human ability. 

I have provided a simple annotation tool in a file called `tools.py`. Make sure that you have a copy of this file and that it is stored in the same directory as this notebook. 

The tool will display images from the evaluation set and provide you with a set of buttons that you can use to decide the orientation of the face. The tool will then display the next image. The tool will continue until you choose the 'QUIT' button. The results of your experiment will be saved in a csv file.

The tool uses some external dependency that you may need to install using `pip`. This can be done by running the cell below

In [ ]:
!pip install jupyter_ui_poll ipywidgets

Before running the tool, we first need to make an empty csv file that will be used for saving the results. We will call this file `human.csv`. Run the cell below to perform this step.


In [ ]:
results_file = 'human.csv'
with open(results_file, 'w') as f:
    f.write("n_pixels, index, direction, label\n")

We can now run the annotation tool. We will start by using the 90 pixel sub-images. Run the code below and label about 50 images before hitting 'QUIT'.

In [ ]:
import tools
import joblib
data_eval = joblib.load('eval1.joblib')

# Using the 90 pixel sub-images
data = data_eval[90]
tools.annotate(data['x_test'], data['y_test'], results_file)

A file called 'human.csv' should have appeared in the folder where you are storing the notebook. Have a look at the contents of the folder in an editor. It will show you the label that you selected and the correct label. You can use this to calculate the accuracy of your annotations.

Now labels another 50 images but using the 50 pixel images using the cell below (note, results will be appended to the existing `human.csv` file).

In [ ]:
data = data_eval[50]
tools.annotate(data['x_test'], data['y_test'], results_file)

Finally, run the tool with the 30 pixel sub-images. Some of these might be hard to get right.

In [ ]:
data = data_eval[30]
tools.annotate(data['x_test'], data['y_test'], results_file)

## 3. Runnning Python script from the command line

So far in this module we have been running all our Python code from inside a Jupyter notebook. Jupyter notebooks are very convenient for rapid prototyping, experimentation and demonstrating ideas, but they are less good for training and evaluating models in a reproducible way. For this, it is better to use a Python script that can be run from the command line. This is what you will need to do for the assignment.

The code below shows a Python script that loads a model and evaluates it with the evaluation data. Cut and paste the code below and save it in a file called `my)evaluate.py`. Then open a command line window and run the script using the command `python my_evaluate.py`. You should see the same accuracy as you saw when you ran the code in the notebook.

(Note, if the code does not work, you may need to install the `joblib` library using `pip`, i.e., run `pip install joblib`).

In [ ]:
"""Python script for evaluating a model

Cut and paste this code into a file called `evaluate.py` and run it with the command `python evaluate.py`.
"""
import joblib

def main():
    # Load the model for 50 pixel sub-images
    model = joblib.load('knn.50.joblib')

    # Load the data and select the 50 pixel sub-images
    all_data = joblib.load('eval1.joblib')
    data = all_data[50]
    x_test = data['x_test']
    y_test = data['y_test']

    # Evaluate the model
    percent_correct = model.score(x_test, y_test) * 100
    print(f'The classifier is {percent_correct:.2f}% correct')


if __name__ == "__main__":
    main()


The code above should all be very familiar from the code we have seen in the notebooks. Notice how all the code that you want to run appears inside a function called `main`. This is good practice and avoids having variables defined in the global scope. This top level function can be called anything, but it is conventional to call it `main`. 

Notice how the call to the function `main` occurs at the bottom of the script and is inside an if-statement. This line "if __name__ == '__main__':" is a common pattern using a special variable "__name__" that is automatically set to "__main__" when the script is run from the command line. So this pattern ensures that the code only runs when it is called from the command line. This is useful because it means you can import this code into another script without it running automatically. Do not worry about this too much, but it is good practice to use this pattern.

The code above is not very versatile because it always uses the same model file and the same sub-image size. It would be better if we could pass these as command line arguments. Python has a module called `argparse` that makes this easy. The code below shows how to use this module to pass the model file and the evaluation data as command line arguments. Cut and paste the code below and save it in a file called `my_evaluate2.py`. Then open a command line window and run the script using the command `python my_evaluate2.py knn.50.joblib 50`. You should see the same accuracy as you saw when you ran the code in the notebook.

In [ ]:
"""Evaluate a model on the eval1 dataset.

Requires the eval1.joblib file to be present in the current directory.

Usage:
    python evaluate_model.py <model_file> <n_pixels>

    model_file: Path to the model file.
    n_pixels: Number of pixels used to train the model.
"""
from argparse import ArgumentParser
from joblib import load


def evaluate(model_file, n_pixels):
    """Evaluate a model on the eval1 dataset.

    Args:
        model_file (str): Path to the model file.
        n_pixels (int): Number of pixels used to train the model.
    """

    print(f"Evaluating {model_file} with {n_pixels} pixels")

    model = load(model_file)

    all_data = load(open("eval1.joblib", "rb"))
    data = all_data[n_pixels]
    x_test = data["x_test"]
    y_test = data["y_test"]

    score = model.score(x_test, y_test)
    return score

def main():
    parser = ArgumentParser()
    parser.add_argument("model_file", type=str)
    parser.add_argument("n_pixels", type=int)
    args = parser.parse_args()
    score = evaluate(args.model_file, args.n_pixels)
    print("Score:", score * 100, "%")

if __name__ == "__main__":
    main()


With your new version of the script you can now also evaluate the other models and the other sub-image sizes. For example, to evaluate the 30 pixel model, you would run the command `python my_evaluate2.py knn.30.joblib 30`. 

If you'd trained your own model you could now use the same script. For example, if you'd trained a model called `my_model.joblib` you could evaluate it using the command `python my_evaluate2.py my_model.joblib 50`.

Note in the code above that the `main` method handles the parsing of the commandline arguments and then calls a function called `evaluate` to do the useful work. It is good practice to separate the argument parsing from the rest of the code functionality. This makes the code easier to read and understand. It also means that the code can be re-used in other scripts. For example, we may have another script that has its own commandline parsing but which wants to use the evaluate function. This can be easily done by importing the `my_evaluate2.py` script, e.g. with `import evaluate from my_evaluate2`. This would not be possible is the commandline parsing and evaluate code were all mixed together in the `main` function.

The `argparse` module is very sophisticated and can handle many different types of commandline arguments. For example, it can handle optional arguments, positional arguments, and arguments with default values. It can also handle arguments that are lists of values. It is easy to learn to use as it has a very good [documentation page](https://docs.python.org/3/library/argparse.html) and there are many examples on the web. The following website has some nicely organised examples that start with simple cases and then build up to more complex usage: https://vnetman.github.io/argparse-recipes/2018/04/21/python-3-argparse-recipe-book.html

## 4. Using Latex

For your final report you have been asked to use LaTeX. LaTeX is a typesetting system that is widely used in academia. It is very good for writing technical documents that contain equations and figures. It is also very good for writing documents that contain references to other documents. You will need to use it for your dissertation, so this is a good opportunity to learn how to use it.

We will discuss this more in the lecture session on Monday.

LaTeX can be installed on your own computer, or it can also be used online via the website https://www.overleaf.com/. Overleaf is an excellent LaTeX hosting service that has a lot of documentation and examples. It is free to use, but you will need to create an account. Whether you prefer to use Overleaf or install LaTeX on your own computer is up to you. The advantage of running on your own computer is that you can work offline and it can be faster. The advantage of Overleaf is that it is easier to get started and you can work on your report from any computer with an internet connection.

## 4.1 Installing LaTeX on your own computer

If you want to install LaTeX on your own computer, you can download it from the following website: https://www.latex-project.org/get/. Installation should be very easy for Linux and Mac. LaTeX is also available for Windows, but it is a bit more complicated to install. If you are using Windows, I recommend that you use Overleaf instead.

There is a template for the assignment stored in a zip file called 'COM6018_latex_template.zip' in the assigment directory of the module's git repository. Run 'git pull' to make sure that you have it and then copy it to a new working directory and unzip it. Move into the folder 'COM6018_latex_template'. If you have LaTeX installed correctly then you should be able to compile the template document using the command 'pdflatex report.tex'. This should create a file called 'report.pdf' that you can open in a PDF viewer. If you have any problems, please ask for help or use Overleaf instead (next section)

## 4.2 Using LaTeX onlin with Overleaf

Overleaf is one of several online LaTeX services, others include Papeeria, LaTeX base, CoCalc. Overleaf is the most popular and has the best documentation and examples. It is free to use, but you will need to create an account.

To get started with Overleaf, go to the website https://www.overleaf.com/ and create an account. Then click on the 'New Project' button and select 'Upload Project'. Then upload the file `COM6018_latex_template.zip` that you can find in the assignment directory of the module's git repository.

After uploading the zip file, the project should open and you will see the LaTeX source on the left and the pdf document that it generates on the right. You can edit the source and the pdf will automatically update. 

We will spend time on Monday explaining how to use LaTeX. In the meantime, you can look at the source of the template document and try to work out what it is doing. You can also look at the Overleaf documentation and examples. The following page is a good place to start: https://www.overleaf.com/learn/latex/Learn_LaTeX_in_30_minutes





*Copyright &copy; 2023, 2024 Jon Barker, University of Sheffield. All rights reserved*.